   ## Extract         Transform             and         Load -  Restaurants in San Antonio
###### -------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Extract: 
#### Data Sources : 
####   1. Yellow Pages
##### Information extracted from yellowpages.com to include:
- Restaurant name <br>
- Restaurant phone number 
- Restaurant website 
- Restaurant type (category) 

#### 2. Google geocode API
##### Information extracted from google API to include:
- Restaurant Address
- Restaurant Latitude
- Restaurant Longitude

In [1]:
# Import dependencies for web scraping
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# Import dependencies for Google API
import requests
import json
from requests.utils import requote_uri
from sqlalchemy import create_engine
import re
# Import api and postgresql keys
from api_keys import g_key, post_key

In [2]:
# Create engine connection and Initialize browser 
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\ubc\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
# Set Base url 
search_url = 'https://www.yellowpages.com/search?search_terms=restaurants&geo_location_terms=San+Antonio%2C+TX'
browser.visit(search_url)

In [4]:
# Create a BeautifulSoup object to contain html page
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

 ### Extraction #1 :
 
 ######    -  Identify the elements by their classes to extract required data and generate a list.
 ######    -  Create a dataframe

In [5]:
# Getting the restaurant info from website and create lists to append information to
rest_info = soup.find_all('div', class_='info')
# Initializing the lists
restaurant = []
phone =[]
website =[]
category =[]

for info in rest_info:
    try:
        restaurant.append(info.find('a', class_="business-name").text)
        phone.append(info.find('div', class_="phone").text)    
        website.append(info.find('a', class_="track-visit-website")['href'])
        category.append(info.find('div', class_="categories").text)       
    except TypeError:
        website.append(float('NAN'))
        category.append(float('NAN'))

#### Data, if not found, will be stored as a _NAN_


###### Note:  A NaN (Not-a-Number) is a symbolic entity encoded in floating-point format. 

In [6]:
# close browser
browser.quit()

In [7]:
# Load information into dataframe
restaurant_df  = pd.DataFrame ( { 'Restaurant_name': restaurant,
                               'Phone_number': phone,
                               'Website': website,
                                'Restaurant_type': category
                             })
restaurant_df.head()

,Restaurant_name,Phone_number,Website,Restaurant_type
0,Jacala Mexican Restaurant,(210) 591-2039,NaN,NaN
1,Marie Callender's,(210) 680-4257,http://www.mariecallenders.com,RestaurantsAmerican RestaurantsBakeries
2,Fujiya Japanese Garden,(210) 966-1930,http://www.fujiyajapanesegardens.com,RestaurantsTake Out RestaurantsJapanese Restau...
3,Golden Wok,(210) 615-8282,NaN,NaN
4,Scuzzi's Italian Restaurant,(210) 742-8677,http://scuzzisitalianrestaurant.com,RestaurantsBarsCocktail Lounges


## Transform #1:

#### From the above table it's clear that data has missing and inaccurate values.

#### Data is cleaned to remove:
- "Restaurants" from the Restaurant_type column 
- "NaN" (missing) values 
- Duplicates 
- _"id" column added and set as index_ to uniquely identify the Restaurants.

##### Clean column Restaurant_type to to get accurate data:

In [8]:
# Remove "Restaurants" tag from Restaurant_type tags
restaurant_df['Restaurant_type'] = restaurant_df.Restaurant_type.str.replace('Restaurants',"")
restaurant_df.head()

,Restaurant_name,Phone_number,Website,Restaurant_type
0,Jacala Mexican Restaurant,(210) 591-2039,NaN,NaN
1,Marie Callender's,(210) 680-4257,http://www.mariecallenders.com,American Bakeries
2,Fujiya Japanese Garden,(210) 966-1930,http://www.fujiyajapanesegardens.com,Take Out Japanese
3,Golden Wok,(210) 615-8282,NaN,NaN
4,Scuzzi's Italian Restaurant,(210) 742-8677,http://scuzzisitalianrestaurant.com,BarsCocktail Lounges


###### Drop Missing values:

In [9]:
# Remove NaN values
restaurant_clean = restaurant_df.dropna(how = 'any')

###### Drop Duplicates:

In [10]:
# Dropping any duplicates
restaurant_clean = restaurant_clean.drop_duplicates(ignore_index=True)

##### Add id column:

In [11]:
# Load all clean information into dataframe, add "id" column and rearrange columns
restaurant_clean_df = restaurant_clean.copy()
restaurant_clean_df['id'] = restaurant_clean.index
restaurant_info = restaurant_clean_df [['id', 'Restaurant_name', 'Website', 'Phone_number', 'Restaurant_type']]
restaurant_info.head()

,id,Restaurant_name,Website,Phone_number,Restaurant_type
0,0,Marie Callender's,http://www.mariecallenders.com,(210) 680-4257,American Bakeries
1,1,Fujiya Japanese Garden,http://www.fujiyajapanesegardens.com,(210) 966-1930,Take Out Japanese
2,2,Scuzzi's Italian Restaurant,http://scuzzisitalianrestaurant.com,(210) 742-8677,BarsCocktail Lounges
3,3,India Taj Palace Indian Restaurant,http://www.indiatajpalace.com,(210) 497-4800,Take Out Indian
4,4,Jingu House at The Japenese Tea Garden,http://jinguhousesa.com,(210) 735-4647,Party PlanningMexican


#### Check to see if dataframe needs any datatype conversion before loading it to the database:

In [12]:
# check to see if dataframe datatype matches database schema
restaurant_info.dtypes

id                  int64
Restaurant_name    object
Website            object
Phone_number       object
Restaurant_type    object
dtype: object

#### Note : datatype matches with the schema generated

## Extraction #2
#### Information extracted from 2nd data source, google api, to include:
- Address of restaurant <br>
- Restaurant latitude <br>
- Restaurant longitude <br>

#### Restaurant location data will be extracted using the restaurant names scraped from 'Yellow Pages'

In [13]:
# Extract location using Restaurant names extracted from Yellow Pages
restaurant_list = restaurant_clean_df['Restaurant_name'].tolist()
#Total number of restaurants
pd.DataFrame({'Total Restaurants': [len(restaurant_list)]})

,Total Restaurants
0,26


In [14]:
# Create lists to store data
lat =[]
lng= []
address =[]


# Set Google geocode API endpoint
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

#####  - Make a request by adding restaurant name as a query parameter to API endpoint
####  - Fetch location data and create a dataframe

In [15]:
# Iterate through restaurant list and get lat,lng and address info
for restaurant in restaurant_list:
    rest = f"{restaurant}, San Antonio, TX"
    # set parameters
    params = {"address": rest, "key": g_key}
    # make a request to API endpoint with restaurant address 
    response = requests.get(base_url, params=params)
    # return request in json format
    rest_go = response.json()
    #extract information
    lat.append(rest_go["results"][0]["geometry"]["location"]["lat"])
    lng.append(rest_go["results"][0]["geometry"]["location"]["lng"])
    address.append(rest_go["results"][0]["formatted_address"])


In [16]:
# Load information into dataframe
location_df = pd.DataFrame ({ 'Lat': lat,
                              'Lng': lng,
                               'Address': address
                            
                        })
location_df.head()

,Lat,Lng,Address
0,29.488611,-98.577500,"4788 Northwest Loop 410, San Antonio, TX 78229..."
1,29.525875,-98.566506,"9030 Wurzbach Rd, San Antonio, TX 78240, USA"
2,29.666083,-98.633976,"24165 I-10 W, Unit 433, San Antonio, TX 78257,..."
3,29.627662,-98.493681,"20323 Huebner Rd, San Antonio, TX 78258, USA"
4,29.460833,-98.476944,"3853 N St Mary's St, San Antonio, TX 78212, USA"


In [17]:
# Total restaurant Locations
pd.DataFrame({'Total Restaurants Locations': [len(location_df)]})

,Total Restaurants Locations
0,26


## Transform #2
#### Data is cleaned to:
- Add "id" column <br>
- Rearrange columns 
- Missing values

In [18]:
# 'id' column added and set as index 
location_df['id'] = location_df.index

restaurant_location = location_df[['id','Address', 'Lat', 'Lng']]
restaurant_location.head()

,id,Address,Lat,Lng
0,0,"4788 Northwest Loop 410, San Antonio, TX 78229...",29.488611,-98.577500
1,1,"9030 Wurzbach Rd, San Antonio, TX 78240, USA",29.525875,-98.566506
2,2,"24165 I-10 W, Unit 433, San Antonio, TX 78257,...",29.666083,-98.633976
3,3,"20323 Huebner Rd, San Antonio, TX 78258, USA",29.627662,-98.493681
4,4,"3853 N St Mary's St, San Antonio, TX 78212, USA",29.460833,-98.476944


##### Check for missing values:

In [19]:
# count total Null values
restaurant_location.isna().sum()

id         0
Address    0
Lat        0
Lng        0
dtype: int64

##### Note: Data is clean and has no missing values 



#### Check for data types:

In [20]:
# check to see if dataframe datatype matches database schema
restaurant_location.dtypes

id           int64
Address     object
Lat        float64
Lng        float64
dtype: object

#### Note: Data types match with the database schema.




## Load: 
### Transformed  data will be loaded to PostgreSQL database.


#### Create engine connection to PostgreSQL:

In [21]:
# create engine connection
rds_connection_string = f"{post_key}:{post_key}@localhost:5432/restaurants_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [22]:
# Confirm tables in postgresql database
engine.table_names()

['restaurant_info', 'restaurant_location']

#### Note: Database schema for above tables have been generated and saved as schema.sql

In [23]:
# Dataframe loaded into database
restaurant_info.to_sql(name='restaurant_info', con=engine, if_exists='replace', index=False)

In [24]:
# Dataframe loaded into database
restaurant_location.to_sql(name='restaurant_location', con=engine, if_exists='replace', index=False)